In [ ]:
# Link do wikipedia - https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil

In [1]:
# Bibliotecas (importar agora, pois eu vou esquecer)
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [2]:
# Abrindo a página do Wikipedia
url = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil"
html = urlopen(url)

In [4]:
# Criando o objeto do BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [11]:
# Criando o padrão de expressão regular para obter os dados a partir das siglas
padrao_estado = re.compile(" \([A-Z]{2}\)")

In [22]:
for linha in soup.find_all("li"):
    if padrao_estado.search(linha):
        print(linha)
    break

TypeError: expected string or bytes-like object